In [ ]:
from pyspark.sql.functions import col, current_date, lit, dayofweek, month, year, when
from datetime import datetime, timedelta

# Read Silver Layer table 
df = spark.read.table("Silver_Layer")

# Converting weather_code to weather_description
df_converted = df.withColumn(
    "weather_description",
    when(col("weather_code") == 0, "Clear sky")
    .when(col("weather_code") == 1, "Mainly clear")
    .when(col("weather_code") == 2, "Partly cloudy")
    .when(col("weather_code") == 3, "Overcast")
    .when(col("weather_code") == 45, "Fog")
    .when(col("weather_code") == 48, "Depositing rime fog")
    .when(col("weather_code") == 51, "Drizzle: light intensity")
    .when(col("weather_code") == 53, "Drizzle: moderate intensity")
    .when(col("weather_code") == 55, "Drizzle: dense intensity")
    .when(col("weather_code") == 56, "Freezing drizzle: light intensity")
    .when(col("weather_code") == 57, "Freezing drizzle: dense intensity")
    .when(col("weather_code") == 61, "Rain: slight intensity")
    .when(col("weather_code") == 63, "Rain: moderate intensity")
    .when(col("weather_code") == 65, "Rain: heavy intensity")
    .when(col("weather_code") == 66, "Freezing rain: light intensity")
    .when(col("weather_code") == 67, "Freezing rain: heavy intensity")
    .when(col("weather_code") == 71, "Snowfall: slight intensity")
    .when(col("weather_code") == 73, "Snowfall: moderate intensity")
    .when(col("weather_code") == 75, "Snowfall: heavy intensity")
    .when(col("weather_code") == 77, "Snow grains")
    .when(col("weather_code") == 80, "Rain showers: slight")
    .when(col("weather_code") == 81, "Rain showers: moderate")
    .when(col("weather_code") == 82, "Rain showers: violent")
    .when(col("weather_code") == 85, "Snow showers: slight")
    .when(col("weather_code") == 86, "Snow showers: heavy")
    .when(col("weather_code") == 95, "Thunderstorm: slight or moderate")
    .when(col("weather_code") == 96, "Thunderstorm with slight hail")
    .when(col("weather_code") == 99, "Thunderstorm with heavy hail")
    .otherwise("Unknown")
)

# Processing weather data and adding weekday and month 
df_gold = (
    df_converted
    .select(
        col("city"),
        col("latitude"),
        col("longitude"),
        col("date"),
        col("temp_max"),
        col("temp_min"),
        col("temp_mean"),
        col("rain_prob"),
        col("weather_code"),
        col("weather_description")
    )
    .withColumn("weekday", dayofweek("date"))         
    .withColumn("month", month("date"))
    .withColumn("year", year(col("date")))               
)

# Overwrite overlapping forecast data
start_dt = datetime.strptime(start_date, "%Y-%m-%d")
end_dt = datetime.strptime(end_date, "%Y-%m-%d")

date_range = [(start_dt + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end_dt - start_dt).days + 1)]

if spark.catalog.tableExists("Gold_Layer"):
    for d in date_range:
        spark.sql(f"DELETE FROM Gold_Layer WHERE date = '{d}'")

# Write Gold Layer result to lakehouse (as table)
df_gold.write.mode("append").format("delta").saveAsTable("Gold_Layer")

# Display result
display(df_gold)

In [1]:
df = spark.sql("SELECT * FROM weather_lakehouse.gold_layer LIMIT 1000")
display(df)

StatementMeta(, 47fae093-b398-4ba8-9999-6a2dcf71860e, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 186926a7-3866-4c2d-b432-39278aefd5ee)